In [2]:
pip install SnowNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SnowNLP: filename=snownlp-0.12.3-py3-none-any.whl size=37760944 sha256=586b372131469558d2127ec2c96ab6458e0f918edd39bf2c040243f7bc0a65a3
  Stored in directory: /root/.cache/pip/wheels/43/f3/70/8990fc249efeb396007766676706f71dd3d1ca3c023ce522ce
Successfully built SnowNLP


In [11]:
import pandas as pd
import jieba
import jieba.analyse
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from snownlp import SnowNLP
import gensim
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Yifan Analysis 1: Keyword and Word Frequency Analysis
def yifan_process_file(file_path, column_name):
    data = pd.read_csv(file_path)
    text = ' '.join(data[column_name].dropna())
    # Chinese text analysis
    if 'Chinese' in file_path:
        keywords = jieba.analyse.extract_tags(text, topK=10)
        words = jieba.lcut(text)
        word_freq = Counter(words).most_common(10)
    else:
        # English text analysis
        vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
        tfidf_matrix = vectorizer.fit_transform([text])
        feature_array = vectorizer.get_feature_names_out()
        tfidf_sorting = tfidf_matrix.max(axis=0).toarray()[0].argsort()[::-1]
        keywords = feature_array[tfidf_sorting][:10]
        word_freq = pd.Series(text.split()).value_counts().head(10)

    return {"Top 10 Keywords": keywords, "Top 10 Word Frequency": word_freq}

# Yifan Analysis 2: Time Series Analysis
def yifan_time_series_analysis(file_path):
    data = pd.read_csv(file_path)
    if 'pubDate' in data.columns:
        data['pubDate'] = pd.to_datetime(data['pubDate'])
        monthly_counts = data.groupby(data['pubDate'].dt.to_period('M')).size()
        return monthly_counts
    else:
        print(f"'pubDate' column not found in {file_path}")
        return None

# Yifan Analysis 3: Text Similarity Analysis
def yifan_calculate_similarity(file_path, text_column):
    data = pd.read_csv(file_path).dropna(subset=[text_column])
    texts = data[text_column]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return cosine_similarity(tfidf_matrix)

# Yifan Analysis 4: Sentiment Analysis
def yifan_perform_sentiment_analysis(file_path, text_column, is_chinese):
    data = pd.read_csv(file_path).dropna(subset=[text_column])
    sentiments = [SnowNLP(text).sentiments if is_chinese else TextBlob(text).sentiment.polarity for text in data[text_column]]
    data['sentiment'] = sentiments
    return data

# Yifan Analysis 5: Topic Modeling Analysis
def yifan_topic_modeling_analysis(file_path, text_column):
    data = pd.read_csv(file_path).dropna(subset=[text_column])
    texts = data[text_column].apply(lambda x: word_tokenize(x.lower()))
    stop_words = set(stopwords.words('english'))
    texts = texts.apply(lambda x: [word for word in x if word not in stop_words])
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    return lda_model.print_topics(-1)

# Yifan Analysis 6: Language Style Analysis
def yifan_analyze_text_style(file_path, text_column):
    data = pd.read_csv(file_path)
    text = ' '.join(data[text_column].dropna())
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    words = [word for word in words if word not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lexical_richness = len(set(words)) / len(words)
    avg_sentence_length = len(words) / len(sentences)
    freq_dist = FreqDist(words)
    pos_tags = nltk.pos_tag(words)
    pos_counts = nltk.FreqDist(tag for (word, tag) in pos_tags)

    # Prepare the data for DataFrame
    data_for_df = {
        "lexical_richness": [lexical_richness],
        "avg_sentence_length": [avg_sentence_length],
        "most_common_words": [dict(freq_dist.most_common(10))],
        "pos_distribution": [dict(pos_counts)]
    }

    return data_for_df


# Main execution block，very important
files_columns = [
    ('/content/preprocessed_Chinese_version_of_Google_News_crawling_yifan.csv', 'text_processed', True),
    ('/content/preprocessed_English_version_of_Google_News_crawling_yifan.csv', 'text_processed', False),
    ('/content/preprocessed_Wikipedia_Chinese_version_of_the_Palestinian-Israeli_conflict_yifan.csv', 'text_processed', True),
    ('/content/preprocessed_Wikipedia_English_version_of_the_Palestinian-Israeli_conflict_yifan.csv', 'text_processed', False)
]

for file_path, text_column, is_chinese in files_columns:
    keyword_freq_data = yifan_process_file(file_path, text_column)  # Analysis 1
    pd.DataFrame(keyword_freq_data).to_csv(file_path.replace("preprocessed_", "keyword_freq_analysis_"), index=False)

    monthly_counts = yifan_time_series_analysis(file_path)  # Analysis 2
    if monthly_counts is not None:
        monthly_counts.to_csv(file_path.replace("preprocessed_", "time_series_analysis_"), index=True)

    similarity_matrix = yifan_calculate_similarity(file_path, text_column)  # Analysis 3
    pd.DataFrame(similarity_matrix).to_csv(file_path.replace("preprocessed_", "similarity_analysis_"), index=False)

    sentiment_data = yifan_perform_sentiment_analysis(file_path, text_column, is_chinese)  # Analysis 4
    sentiment_output_file = file_path.replace("preprocessed_", "sentiment_analysis_")
    sentiment_data.to_csv(sentiment_output_file, index=False)

    topics = yifan_topic_modeling_analysis(file_path, text_column)  # Analysis 5
    pd.DataFrame({"Topics": topics}).to_csv(file_path.replace("preprocessed_", "topic_modeling_analysis_"), index=False)

    style_analysis = yifan_analyze_text_style(file_path, text_column)  # Analysis 6
    pd.DataFrame(style_analysis, index=[0]).to_csv(file_path.replace("preprocessed_", "style_analysis_"), index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
<ipython-input-11-654e1178499f>:49: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  monthly_counts = data.groupby(data['pubDate'].dt.to_period('M')).size()
<ipython-input-11-654e1178499f>:49: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  monthly_counts = data.groupby(data['pubDate'].dt.to_period('M')).size()


'pubDate' column not found in /content/preprocessed_Wikipedia_Chinese_version_of_the_Palestinian-Israeli_conflict_yifan.csv
'pubDate' column not found in /content/preprocessed_Wikipedia_English_version_of_the_Palestinian-Israeli_conflict_yifan.csv
